In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=72be8bbe-9e6f-4462-854c-a98744515794
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:05<00:00, 242MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from models import attention_based_CNN
from models import CompModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 7868.42it/s]


In [6]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)
# transformation and resizing
x_train = RP_final_data(x_train)

100%|██████████| 10344/10344 [00:00<00:00, 826429.21it/s]
10344it [00:08, 1231.55it/s]
10344it [00:04, 2132.39it/s]
100%|██████████| 5000/5000 [24:36<00:00,  3.39it/s]


In [7]:
model = attention_based_CNN()
model, lr_sch = CompModel(model)

history = model.fit(x_train, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100
157/157 [==============================] - 28s 109ms/step - loss: 4.8076 - acc: 0.4530 - lr: 0.0010
Epoch 2/100
157/157 [==============================] - 17s 108ms/step - loss: 2.0156 - acc: 0.6252 - lr: 9.7724e-04
Epoch 3/100
157/157 [==============================] - 17s 108ms/step - loss: 1.3364 - acc: 0.7038 - lr: 9.5499e-04
Epoch 4/100
157/157 [==============================] - 17s 108ms/step - loss: 1.2111 - acc: 0.7554 - lr: 9.3325e-04
Epoch 5/100
157/157 [==============================] - 17s 108ms/step - loss: 0.9773 - acc: 0.8012 - lr: 9.1201e-04
Epoch 6/100
157/157 [==============================] - 17s 108ms/step - loss: 0.7525 - acc: 0.8370 - lr: 8.9125e-04
Epoch 7/100
157/157 [==============================] - 17s 108ms/step - loss: 0.5820 - acc: 0.8500 - lr: 8.7096e-04
Epoch 8/100
157/157 [==============================] - 17s 108ms/step - loss: 0.6635 - acc: 0.8602 - lr: 8.5114e-04
Epoch 9/100
157/157 [==============================] - 17s 108ms/step - loss

In [8]:
model.save('/kaggle/working/RP_atten_model.h5')